In [1]:
import pyBach
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

<H1>Scraping Bachelor(ette) Episode and Reddit Data</H1>
pyBach is a collection of functions that allows for scraping of bachelor and bachelorette data from Wikipedia and Reddit
<ol>
    <li>First scrape last 5 season of The Bachelor and The Bachelorette into a pandas dataframe</li>
    <li>Join all individual dataframes into one large dataframe</li>

In [2]:
dfList = []
bachSeasons = range(19,24)
etteSeasons = range(10,15)

for bachSeason in bachSeasons:
    dfList.append(pyBach.scrapeWikiEpiTable(bachSeason))

for etteSeason in etteSeasons:
    dfList.append(pyBach.scrapeWikiEpiTable(etteSeason, show='bachelorette'))

df = pd.concat(dfList)

The dataframe contains the following columns:

In [3]:
print(df.columns.values)

['numOverall' 'numInSeason' 'year' 'month' 'day' 'posix time' 'isBachelor'
 'season' 'viewers(millions)']


In [4]:
df = pyBach.appendStartTime(df) #default value is 6 days prior to episode airdate
df = pyBach.appendEndTime(df) #default value 0 days

In [5]:
df.head()

,numOverall,numInSeason,year,month,day,posix time,isBachelor,season,viewers(millions),startEpoch,endEpoch
0,182.0,1.0,2015,1,5,1420434000,1.0,19,7.76,1419933600,1420452000
1,183.0,2.0,2015,1,12,1421038800,1.0,19,6.48,1420538400,1421056800
2,184.0,3.0,2015,1,19,1421643600,1.0,19,7.61,1421143200,1421661600
3,185.0,4.0,2015,1,26,1422248400,1.0,19,7.95,1421748000,1422266400
4,186.0,5.0,2015,2,2,1422853200,1.0,19,8.45,1422352800,1422871200


<H1>Bachelor and Bachelorette Views per Season</H1>
Since the goal of this analysis is to predict if a show will have high ratings based on the previous weeks Reddit posts, we first need to determine if we need to correct for any overall trends.

Since the after the final rose and (wo)men tell all are a different format, lets filter them out of our dataframe

In [6]:
df = pyBach.filterData(df)

AttributeError: module 'pyBach' has no attribute 'filterData'

In [ ]:
fig, (ax1, ax2) = plt.subplots(ncols=2, sharey=True)
fig.set_figwidth(20)
dfBach = df[df['show'] == 'bachelor']
dfEtte = df[df['show'] == 'bachelorette']
sns.barplot(x=dfBach['season'], y=dfBach['u.s. viewers(millions)'], ax=ax1, ci=95)
sns.barplot(x=dfEtte['season'], y=dfEtte['u.s. viewers(millions)'], ax=ax2, ci=95)
ax1.set_title('The Bachelor')
ax2.set_title('The Bachelorette')

sns.set(style='whitegrid')


In [ ]:
#Run a one-way anova on views to be sure
import scipy.stats as stats

f, p = stats.f_oneway(*[dfBach.loc[dfBach.season == x, 'u.s. viewers(millions)'] for x in range(19,24)])
print('The Bachelor Season''s one-way ANOVA P-value is:' + str(p) + '\n')
f, p = stats.f_oneway(*[dfEtte.loc[dfEtte.season == x, 'u.s. viewers(millions)'] for x in range(10,15)])
print('The Bachelorette Season''s one-way ANOVA P-Value is:' + str(p) + '\n')

In [ ]:
fig, axes = plt.subplots(ncols=5, sharey=True)
fig.set_figwidth(15)
fig.suptitle("The Bachelor", fontsize=16)
idx = 0
for subAx in axes:
    sns.barplot(x=dfBach.loc[dfBach.season == bachSeasons[idx], 'no. inseason'], y=dfBach.loc[dfBach.season == bachSeasons[idx], 'u.s. viewers(millions)'], ax=subAx)
    idx = idx + 1
    

In [ ]:
fig, axes = plt.subplots(ncols=5, sharey=True)
fig.set_figwidth(15)
fig.suptitle("The Bachelorette", fontsize=16)
idx = 0
for subAx in axes:
    sns.barplot(x=dfEtte.loc[dfEtte.season == etteSeasons[idx], 'no. inseason'], y=dfEtte.loc[dfEtte.season == etteSeasons[idx], 'u.s. viewers(millions)'], ax=subAx)
    idx = idx + 1

To maximize the possibility of detecting a difference in social media buzz, I am going to limit my initial analysis to only the worse performing and best performing episodes of each season.

In [ ]:
# normalize each episode's viewer number by the season finale
df.loc[:, 'norm views'] = pd.Series([0]*df.shape[0])
for showStr in ['bachelor', 'bachelorette']:
    if showStr == 'bachelor':
        for season in bachSeasons:
            df.loc[
                (df.season==season) & (df.show==showStr),'norm views'] = df.loc[(df.season==season) & (df.show==showStr), 'u.s. viewers(millions)']\
                /df.loc[(df.season==season) & (df.show==showStr), 'u.s. viewers(millions)'].max()
    else:
        for season in etteSeasons:
            df.loc[
                (df.season==season) & (df.show==showStr), 'norm views'] = df.loc[(df.season==season) & (df.show==showStr), 'u.s. viewers(millions)']\
            /df.loc[(df.season==season) & (df.show==showStr), 'u.s. viewers(millions)'].max()

In [ ]:
# Remove all the season finales whose norm views are one
dfFinless = df.loc[~(df['norm views']==1),:]

In [ ]:
# pull out the largest norm views for each season of each show
minList = []
maxList = []
for showStr in ['bachelor', 'bachelorette']:
    if showStr == 'bachelor':
        for season in bachSeasons:
            minValue = dfFinless.loc[(dfFinless.season == season) & (dfFinless.show==showStr), 'norm views'].min()
            tempMin = dfFinless.loc[(dfFinless.season==season) & (dfFinless.show==showStr) & (dfFinless['norm views']==minValue),:]
            # we only want ONE min, so if more than one row is found, take the earlier episode
            if tempMin.shape[0] == 1:
                minList.append((tempMin))
            else:
                minList.append((tempMin.loc[tempMax['no. inseason']==tempMin['no. inseason'].min(),:]))
            maxValue = dfFinless.loc[(dfFinless.season == season) & (dfFinless.show==showStr), 'norm views'].max()
            tempMax = dfFinless.loc[(dfFinless.season==season) & (dfFinless.show==showStr) & (dfFinless['norm views']==maxValue), :]
            if tempMax.shape[0] == 1:
                maxList.append((tempMax))
            else:
                maxList.append((tempMax.loc[tempMax['no. inseason']==tempMax['no. inseason'].min(),:]))
    else:
        for season in etteSeasons:
            minValue = dfFinless.loc[(dfFinless.season == season) & (dfFinless.show==showStr), 'norm views'].min()
            tempMin = dfFinless.loc[(dfFinless.season==season) & (dfFinless.show==showStr) & (dfFinless['norm views']==minValue),:]
            if tempMin.shape[0] == 1:
                minList.append((tempMin))
            else:
                minList.append((tempMin.loc[tempMax['no. inseason']==tempMin['no. inseason'].min(),:]))
            maxValue = dfFinless.loc[(dfFinless.season == season) & (dfFinless.show==showStr), 'norm views'].max()
            tempMax = dfFinless.loc[(dfFinless.season==season) & (dfFinless.show==showStr) & (dfFinless['norm views']==maxValue), :]
            if tempMax.shape[0] == 1:
                maxList.append((tempMax))
            else:
                maxList.append((tempMax.loc[tempMax['no. inseason']==tempMax['no. inseason'].min(),:]))

In [ ]:
# concat lists into one dataframe
dfMin = pd.concat(minList)
dfMax = pd.concat(maxList)

In [ ]:
dfMax

In [ ]:
dfMin

In [ ]:
# collect reddit submission list for min and max dfs
subListsMax = []
subListsMin = []
for idx, row in dfMax.iterrows():
    subListsMax.append(pyBach.getRedditSubsList(row['startEpoch'], row['endEpoch'])) #use my default PRAW User Agent
dfMax['subList'] = subListsMax
for idx, row in dfMin.iterrows():
    subListsMin.append(pyBach.getRedditSubsList(row['startEpoch'], row['endEpoch']))
dfMin['subList'] = subListsMin

In [ ]:
dfMax.head()

In [ ]:
dfMin.head()

In [ ]:
import pickle as pk

In [ ]:
dfMin.to_pickle('dfMin.pickle')
dfMax.to_pickle('dfMax.pickle')

In [ ]:
print(subListsMin[0])